In [1]:
from binance.client import Client
import pandas as pd
import time
from datetime import datetime, timedelta

# Initialize Binance Futures client
client = Client()

#start_time and end_time in milliseconds
#This function fetches total traded volume
def get_agg_trades(symbol, start_time, end_time):
    
    trades = []
    while start_time < end_time:
        batch = client.futures_aggregate_trades(   #Fetch aggregate trades
            symbol=symbol,
            startTime=start_time,
            endTime=end_time,
            limit=1000                     #max allowed trades in one request
        )
        if not batch:
            break
        trades.extend(batch)               #store the trades
        last_trade_time = batch[-1]['T']   #last trade time
        start_time = last_trade_time + 1   #start time
        time.sleep(0.1)                    # To prevent rate limits
    return trades


###seprate the bid and ask volume from the trades
def process_trades_to_volume_df(trades, interval='1min'):
    
    records = []
    for trade in trades:
        ts = pd.to_datetime(trade['T'], unit='ms')       #data frame timestamp
        qty = float(trade['q'])                          #Total quantity traded in 1min
        is_buyer_maker = trade['m']                      #m -> buy maker flag
        bid_volume = qty if is_buyer_maker else 0        #bid
        ask_volume = qty if not is_buyer_maker else 0    #ask
        records.append({                                 
            'timestamp': ts,
            'bid_volume': bid_volume,
            'ask_volume': ask_volume
        })

    df = pd.DataFrame(records)                           #data frame
    df.set_index('timestamp', inplace=True)              #set index to timestamp
    df = df.resample(interval).sum().fillna(0)           #Resample is  important for interval(add every bid/ask volume in 1min)
    df['total_volume'] = df['bid_volume'] + df['ask_volume']
    return df

# === USER PARAMETERS ===
symbol = 'SOLUSDT'
start_date = '2025-01-01 00:00:00'
end_date   = '2025-08-19 00:00:00'
interval = '1H'

# === TIME CONVERSION ===
start_ts = int(pd.to_datetime(start_date).timestamp() * 1000)
end_ts = int(pd.to_datetime(end_date).timestamp() * 1000)

# === FETCH AND PROCESS ===
print("Downloading aggregate trades...")
agg_trades = get_agg_trades(symbol, start_ts, end_ts)
print(f" Fetched {len(agg_trades)} trades.")

print("Aggregating volumes...")
volume_df = process_trades_to_volume_df(agg_trades, interval=interval)

# === EXPORT ===
volume_df.to_csv(f'{symbol}_bid_ask_volume_{interval}_3_4.csv')
print(volume_df.head())


BinanceAPIException: APIError(code=-1003): Too many requests; current limit of IP(103.253.174.244) is 2400 requests per minute. Please use the websocket for live updates to avoid polling the API.

## New

In [6]:
from binance.client import Client
import pandas as pd

client = Client()

symbol = "SOLUSDT"
interval = Client.KLINE_INTERVAL_1HOUR   # 1H candles
start_date = "2025-01-01 00:00:00"
end_date   = "2025-02-01 00:00:00"

klines = client.futures_klines(
    symbol=symbol,
    interval=interval,
    startTime=int(pd.to_datetime(start_date).timestamp() * 1000),
    endTime=int(pd.to_datetime(end_date).timestamp() * 1000)
)

# Convert to DataFrame
df = pd.DataFrame(klines, columns=[
    "open_time","open","high","low","close","volume",
    "close_time","quote_asset_volume","num_trades",
    "taker_buy_base","taker_buy_quote","ignore"
])

# Convert types
df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")
df["volume"] = df["volume"].astype(float)
df["taker_buy_base"] = df["taker_buy_base"].astype(float)

# Compute bid/ask volumes
df["bid_volume"] = df["taker_buy_base"]         # taker is buyer → bid
df["ask_volume"] = df["volume"] - df["bid_volume"]
df["total_volume"] = df["volume"]

print(df[["open_time","bid_volume","ask_volume","total_volume"]])
df.to_csv("SOLUSDT_1H_bid_ask_volume.csv", index=False)


              open_time  bid_volume  ask_volume  total_volume
0   2025-01-01 00:00:00    359295.0    227245.0      586540.0
1   2025-01-01 01:00:00    152790.0    182184.0      334974.0
2   2025-01-01 02:00:00    107411.0     96720.0      204131.0
3   2025-01-01 03:00:00     71486.0    108967.0      180453.0
4   2025-01-01 04:00:00    130134.0    145544.0      275678.0
..                  ...         ...         ...           ...
495 2025-01-21 15:00:00    527910.0    608214.0     1136124.0
496 2025-01-21 16:00:00    899949.0    900045.0     1799994.0
497 2025-01-21 17:00:00    459655.0    409816.0      869471.0
498 2025-01-21 18:00:00    904147.0    878374.0     1782521.0
499 2025-01-21 19:00:00    756503.0    860104.0     1616607.0

[500 rows x 4 columns]
